In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# a1 = ee.FeatureCollection("users/311605001111/region/YangtzeRiver")
# a2 = ee.Geometry.LineString([[119.613014, 32.190599], [119.636902, 32.197143], [119.670852, 32.218884], [119.675845, 32.222581], [119.683646, 32.250301], [119.715476, 32.29701], [119.726085, 32.298065], [119.760099, 32.320751], [119.800041, 32.317586], [119.838735, 32.295427], [119.872558, 32.264089], [119.892529, 32.189104], [119.918117, 32.097138], [119.951042, 32.043468], [119.998474, 31.98101], [120.089592, 31.951354], [120.171974, 31.933345], [120.248114, 31.936523], [120.344225, 31.979951], [120.474038, 32.045584], [120.582632, 32.00748], [120.712445, 32.015949], [120.811052, 31.973597], [120.864725, 31.880354], [120.880952, 31.839], [120.918398, 31.822029], [120.955844, 31.783832]])
# river = a1.geometry().union(a2)
# Map.addLayer(river, {}, "river")

# roi = river.buffer(8000)
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,7)

roi = ee.FeatureCollection('users/311605001111/region/donghu')
# roi = ee.FeatureCollection('users/311605001111/region/liangzihu')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,10)

# l8+l7

## 相关函数

In [ ]:
# 移除无效像元
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5) 
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)
def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)  
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    return image_cloud

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')

    return image.addBands(mndwi).addBands(awei)

In [ ]:
def cal_cloud(image):
    img = image.clip(roi)
    validPixel = maskSR(img).select('pixel_qa').gt(0).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,validPixel]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return img.set({'Clouds_on_water':rate})

def add_property(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('system:index')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'row': image.get('WRS_ROW')})
    point = point.set({'path': image.get('WRS_PATH')})
    point = point.set({'Clouds_on_water': image.get('Clouds_on_water')})
    return point

##  landsat8

In [ ]:
NAME = 'DONGHU'
for year in range(2013,2022,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate,endDate)
    filter_images = ee.FeatureCollection(landsat_images.map(cal_cloud).filter(ee.Filter.lt('Clouds_on_water',50)).map(add_property))
    dataset_id = NAME + '_landsat8_' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filter_images,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

In [ ]:
NAME = 'DONGHU'
for year in range(2021,2022,1):
    dataset_id = 'users/311605001111/'+ NAME + '_landsat8_' + str(year)
    points = ee.FeatureCollection(dataset_id)
    geemap.ee_to_csv(points, NAME + '_landsat8_{}.csv'.format(year))

##  landsat7

In [ ]:
for year in range(2014,2022,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    landsat_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate,endDate)
    filter_images = ee.FeatureCollection(landsat_images.map(cal_cloud).filter(ee.Filter.lt('Clouds_on_water',5)).map(add_property))
    dataset_id = 'CHONGQING_SHANGHAI_landsat7_' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filter_images,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

In [ ]:
for year in range(2014,2022,1):
    dataset_id = 'users/311605001111/CHONGQING_SHANGHAI_landsat7_' + str(year)
    points = ee.FeatureCollection(dataset_id)
    geemap.ee_to_csv(points,'CHONGQING_SHANGHAI_landsat7_{}.csv'.format(year))

# sentinel_2

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)
def maskS2clouds_reverse(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask.remap([0,1],[1,0]))
def cal_cloud_s2(image):
    img = image.clip(roi)
    validPixel = maskS2clouds(img).select('B2').gt(0).rename('waterclass').float()
    invalidPixel = maskS2clouds_reverse(img).select('B2').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,validPixel]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 200,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return img.set({'Clouds_on_water':rate})
def add_property_s2(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('id')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'satellite name': image.get('SPACECRAFT_NAME')})
    point = point.set({'product_id': image.get('PRODUCT_ID')})
    point = point.set({'mgrs_tile': image.get('MGRS_TILE')})
    point = point.set({'Clouds_on_water': image.get('Clouds_on_water')})
    point = point.set({'CLOUD_COVERAGE_ASSESSMENT': image.get('CLOUD_COVERAGE_ASSESSMENT')})
    return point

In [ ]:
# sentinel_images = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(roi).filterDate('2020-01-01', '2020-12-31')
# visualization = {'min': 0,'max': 5000,'bands': ['B8', 'B4', 'B3']}
# Map.addLayer(sentinel_images.map(maskS2clouds).median(), visualization, 'RGB')

In [ ]:
# for year in range(2015,2022,1):
#     print(year)
#     startDate = str(year) + '-01-01'
#     endDate = str(year) + '-12-31'
#     sentinel_images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(startDate,endDate)
#     filter_images = ee.FeatureCollection(sentinel_images.map(cal_cloud_s2).filter(ee.Filter.lt('Clouds_on_water',5)).map(add_property_s2))
#     dataset_id = 'CHONGQING_SHANGHAI_sentinel2_' + str(year)
#     assetID = 'users/311605001111/' + dataset_id
#     task = ee.batch.Export.table.toAsset(**{
#         'collection': filter_images,
#         'description': dataset_id,
#         'assetId': assetID
#     })
#     task.start()
for year in range(2018,2019,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    sentinel_images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(startDate,endDate)
    filter_images = ee.FeatureCollection(sentinel_images.map(cal_cloud_s2).map(add_property_s2))
    dataset_id = 'LIANGZIHU_sentinel2_' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filter_images,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

In [ ]:
NAME = 'LIANGZIHU'
for year in range(2018,2019,1):
    dataset_id = 'users/311605001111/' + NAME + '_sentinel2_' + str(year)
    points = ee.FeatureCollection(dataset_id)
    geemap.ee_to_csv(points,NAME + '_sentinel2_{}.csv'.format(year))

# MOD09

In [4]:
def cloud_cover(image):
    qa = image.clip(roi).select('state_1km')
    mask = qa.rightShift(0).mod(4)
    all_pixels = mask.gte(0)
    bad = mask.gt(0)
    count_allPixels = all_pixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 1000,'maxPixels': 1e14,})
    count_bad = bad.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 1000,'maxPixels': 1e14,})
    cloud_percentage = ee.Number(count_bad.get('state_1km')).divide(ee.Number(count_allPixels.get('state_1km'))).multiply(100)
    return image.set({'Clouds_on_water' : cloud_percentage})

def add_property_mod(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('system:id')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
    point = point.set({'Clouds_on_water': image.get('Clouds_on_water')})
    return point

In [ ]:
NAME = 'LIANGZIHU'
for year in range(2013,2022,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    images = ee.ImageCollection('MODIS/006/MOD09GA').filterBounds(roi).filterDate(startDate,endDate)
    filter_images = ee.FeatureCollection(images.map(cloud_cover,True).filter(ee.Filter.lt('Clouds_on_water',10)).map(add_property_mod,True))
    dataset_id = NAME + '_MOD09GA_' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filter_images,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

In [ ]:
NAME = 'DONGHU'
for year in range(2017,2020,1):
    dataset_id = 'users/311605001111/' + NAME + '_MOD09GA_' + str(year)
    points = ee.FeatureCollection(dataset_id)
    geemap.ee_to_csv(points, NAME + '_MOD09GA_{}.csv'.format(year))

In [5]:
NAME = 'DONGHU'
for year in range(2017,2020,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    images = ee.ImageCollection('MODIS/006/MYD09GA').filterBounds(roi).filterDate(startDate,endDate)
    filter_images = ee.FeatureCollection(images.map(cloud_cover).filter(ee.Filter.lt('Clouds_on_water',10)).map(add_property_mod))
    dataset_id = NAME + '_MYD09GA_' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': filter_images,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

2017
2018
2019


In [6]:
NAME = 'DONGHU'
for year in range(2017,2020,1):
    dataset_id = 'users/311605001111/'+ NAME + '_MYD09GA_' + str(year)
    points = ee.FeatureCollection(dataset_id)
    geemap.ee_to_csv(points,NAME + '_MYD09GA_{}.csv'.format(year))

Generating URL ...
Please wait ...
Data downloaded to C:\Users\DELL\gee\DONGHU_MYD09GA_2017.csv
Generating URL ...
Please wait ...
Data downloaded to C:\Users\DELL\gee\DONGHU_MYD09GA_2018.csv
Generating URL ...
Please wait ...
Data downloaded to C:\Users\DELL\gee\DONGHU_MYD09GA_2019.csv


In [ ]:
# for i in range(2015,2017,1):
#     dataset = ee.ImageCollection('MODIS/006/MOD09GA').filter(ee.Filter.date('{}-01-01'.format(i), '{}-12-31'.format(i))).filterBounds(roi).map(cloud_cover)
#     dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
#     points = dataset_filter.map(add_property)
#     geemap.ee_to_csv(ee.FeatureCollection(points),'MOD09GA_{}.csv'.format(i))

# for i in range(2008,2012,1):
#     dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('{}-01-01'.format(i), '{}-12-31'.format(i))).filterBounds(roi).map(cloud_cover)
#     dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
#     points = dataset_filter.map(add_property)
#     geemap.ee_to_csv(ee.FeatureCollection(points),'taihu_MYD09GA_{}.csv'.format(i))

# 下载水体

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2021-01-01','2021-11-30').map(maskSR).median().clip(roi)
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
awei = water_index(l8).select('AWEI').gt(0).selfMask()
Map.addLayer(l8, visParams,'image mosaic')
Map.addLayer(awei,{'palette':['blue']},'awei')

In [ ]:
dataset_id = 'CHONGQING_SHANGHAI'
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': awei,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
image = ee.Image('users/311605001111/CHONGQING_SHANGHAI')
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'CHONGQING_SHANGHAI.tif')
geemap.ee_export_image(image, filename=filename, scale=210, region=roi)

In [ ]:
sentinel_images = ee.ImageCollection('COPERNICUS/S2_SR').select(['B2', 'B3','B4','B8','B11','B12','QA60'],['B2', 'B3','B4','B5', 'B6', 'B7','QA60'])
s2 = sentinel_images.filterBounds(roi).filterDate('2020-01-01', '2020-12-31').map(maskS2clouds).median().clip(roi)
visualization = {'min': 0,'max': 5000,'bands': ['B5', 'B4', 'B2']}
Map.addLayer(s2, visualization, 'RGB')
awei_s2 = water_index(s2).select('AWEI').gt(0)
Map.addLayer(awei_s2.selfMask(),{'palette':['blue']},'awei_s2')


In [ ]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31')

filter_images = landsat_images.map(cal_cloud).filter(ee.Filter.lt('Clouds_on_water',5)).map(add_property)
print(filter_images.size().getInfo())
geemap.ee_to_csv(ee.FeatureCollection(filter_images),'CHONGQING_SHANGHAI_landsat8.csv')

In [ ]:
q = landsat_images.map(cal_cloud).filter(ee.Filter.lt('Clouds_on_water',5))
print(q.aggregate_array('system:index').getInfo())